In [1]:
import SimpleITK as sitk
import matplotlib.pyplot as plt

def resample_to_reference(image, reference_image):
    # Convertir les images en objets SimpleITK
    sitk_image = sitk.GetImageFromArray((image * 255).astype('uint8'))
    sitk_reference_image = sitk.GetImageFromArray((reference_image * 255).astype('uint8'))

    # Utiliser Resample pour redimensionner l'image à la même grille que l'image de référence
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(sitk_reference_image)
    resampled_image = resampler.Execute(sitk_image)

    # Convertir l'image redimensionnée en tableau numpy
    result_image = sitk.GetArrayFromImage(resampled_image)

    return result_image

def mutual_information(image1, image2):
    # Redimensionner les images à la même dimension
    image1_resampled = resample_to_reference(image1, image2)
    image2_resampled = resample_to_reference(image2, image1)

    # Convertir les images en objets SimpleITK
    sitk_image1 = sitk.GetImageFromArray((image1_resampled * 255).astype('uint8'))
    sitk_image2 = sitk.GetImageFromArray((image2_resampled * 255).astype('uint8'))

    # Assurer que les images ont la même taille
    if sitk_image1.GetSize() != sitk_image2.GetSize():
        sitk_image1 = sitk.Resample(sitk_image1, sitk_image2)
    
    # Créer un filtre pour le recalage basé sur l'information mutuelle
    registration_method = sitk.ImageRegistrationMethod()

    # Configurer les paramètres du recalage
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear)

    # Configurer la transformation (rigide dans cet exemple)
    initial_transform = sitk.CenteredTransformInitializer(
        sitk_image1, sitk_image2, sitk.Euler3DTransform()
    )
    registration_method.SetInitialTransform(initial_transform)

    # Configurer les paramètres d'optimisation
    registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
    registration_method.SetOptimizerScalesFromPhysicalShift()

    # Exécuter le recalage
    final_transform = registration_method.Execute(sitk_image1, sitk_image2)

    # Appliquer la transformation à l'image source
    transformed_image = sitk.Resample(sitk_image1, sitk_image2, final_transform, sitk.sitkLinear, 0.0)

    # Convertir l'image transformée en tableau numpy
    result_image = sitk.GetArrayFromImage(transformed_image)

    return result_image

# Exemple d'utilisation
if __name__ == "__main__":
    # Charger les deux images que vous souhaitez recalibrer
    image1 = plt.imread('20230323_191223_BRS_L_UP_11.369_00.png.jpg')
    image2 = plt.imread('20230406_060421_BRS_L_UP_11.369_00.png.jpg')

    # Recalage basé sur l'information mutuelle
    result_image = mutual_information(image1, image2)

    # Afficher les images originales et recalées
    plt.figure(figsize=(10, 5))
    plt.subplot(131), plt.imshow(image1, cmap='gray'), plt.title('Image 1')
    plt.subplot(132), plt.imshow(image2, cmap='gray'), plt.title('Image 2')
    plt.subplot(133), plt.imshow(result_image, cmap='gray'), plt.title('Image Recalée')
    plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '20230323_191223_BRS_L_UP_11.369_00.png.jpg'